Milestone2


In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import pandas as pd
import numpy as np

#kagglehub.login()


Dataset available at: https://www.kaggle.com/datasets/tonygordonjr/football-match-statistics-and-more/data?fbclid=IwZXh0bgNhZW0CMTEAAR27HPyyqC9nUioeOqJzVzznFoIH3SNY0xcdZ_4dDxEpDOoRnZgcaXdTTRE_aem_0dX5wF1-vk6WmlQWZMnQ2g

In [19]:
!kaggle datasets download -d tonygordonjr/football-match-statistics-and-more
!unzip /content/football-match-statistics-and-more.zip -d ./

Dataset URL: https://www.kaggle.com/datasets/tonygordonjr/football-match-statistics-and-more
License(s): other
football-match-statistics-and-more.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  /content/football-match-statistics-and-more.zip
replace ./coaches.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./fixture_events_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./fixture_injuries_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./fixture_lineup_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./fixture_stats_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: nn
replace ./fixture_stats_players_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./fixture_stats_teams_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./season_standings_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./season_stats_teams_clean.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace ./venues.c

In [20]:
teams_clean_col = ['fixture_id', 'team_id', 'team_name', 'fouls','yellow_cards', 'red_cards']
stats_clean_col = ['fixture_id', 'fixture_date', 'fixture_referee', 'league_id', 'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id', 'teams_away_name']
players_clean_col = ['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name', 'fouls_committed', 'yellow_cards', 'red_cards', 'game_minutes']

fix_teams_clean = pd.read_csv("fixture_stats_teams_clean.csv",usecols=teams_clean_col)
fix_clean = pd.read_csv("fixture_stats_clean.csv", usecols=stats_clean_col)
fix_players_clean = pd.read_csv("fixture_stats_players_clean.csv",usecols=players_clean_col)

In [21]:
fix_players_clean[fix_players_clean['player_id'] == 0]
fix_players_clean.player_name.value_counts()

,count
player_name,
Danilo,662
Ben Davies,507
Marcelo,471
Raúl García,462
Fabinho,455
...,...
Tim Hall,1
Andrei Klimovich,1
Alvis Jaunzems,1


In [22]:
fix_players_clean['total_cards'] = fix_players_clean.apply(lambda x: x['yellow_cards'] + x['red_cards'] * 2 if x['yellow_cards'] != 2 else 3, axis=1)
fix_players_clean.sort_values('red_cards', ascending=False).head()

,fixture_id,team_id,team_name,player_id,player_name,game_minutes,fouls_committed,yellow_cards,red_cards,total_cards
98098,12291,36,Fulham,49923,Aboubakar Kamara,63.0,5,0,1,2
1000312,1149546,628,Sparta Praha,295027,Matěj Ryneš,80.0,2,2,1,3
290364,194669,163,Borussia Monchengladbach,1464,Granit Xhaka,86.0,6,2,1,3
132123,571820,80,Lyon,651,Marcelo,90.0,1,0,1,2
963004,592363,48,West Ham,1243,Tomáš Souček,90.0,0,0,1,2


In [23]:
fix_players_clean.drop(['yellow_cards', 'red_cards'], axis=1, inplace=True)

In [24]:
fix_players_clean.head()

,fixture_id,team_id,team_name,player_id,player_name,game_minutes,fouls_committed,total_cards
0,855753,1,Belgium,20,Axel Witsel,90.0,1,0
1,135803,1,Belgium,2296,Eden Hazard,68.0,0,0
2,855753,1,Belgium,1422,Jeremy Doku,NaN,0,0
3,718242,1,Belgium,1422,Jeremy Doku,90.0,2,0
4,132793,1,Belgium,2925,Dennis Praet,1.0,0,0


In [25]:
fix_total_cards = fix_players_clean.groupby('fixture_id')['total_cards'].sum().reset_index()

In [26]:
fix_players_clean.drop(fix_players_clean[fix_players_clean["game_minutes"].isna()].index, inplace=True)

In [27]:
fix_players_clean["game_minutes"].isna().sum()

np.int64(0)

In [28]:
fix_players_clean["game_minutes"].sum()

np.float64(64253242.0)

In [32]:
fix_players_clean_og = fix_players_clean.merge(fix_clean[['fixture_id', 'fixture_date']], on='fixture_id', how='left')

In [34]:
def calculate_avg_cards_per_90(df):
    # Sort by fixture_id to ensure chronological order
    df = df.sort_values(by=['player_name', 'fixture_date'])

    # Create new columns for storing cumulative stats
    df['cumulative_cards'] = 0
    df['cumulative_fouls'] = 0
    df['cumulative_minutes'] = 0
    df['avg_cards_per_90'] = 0.0
    df['avg_fouls_per_90'] = 0.0

    # Group by player_name and iterate to compute averages from past matches
    grouped = df.groupby('player_name')

    for player_name, player_df in grouped:
        cumulative_cards = 0
        cumulative_fouls = 0
        cumulative_minutes = 0

        for idx, row in player_df.iterrows():
            # Compute averages from past matches
            if cumulative_minutes > 0:
                avg_cards = (cumulative_cards / cumulative_minutes) * 90
                avg_fouls = (cumulative_fouls / cumulative_minutes) * 90
            else:
                avg_cards = 0
                avg_fouls = 0

            # Store calculated averages
            df.at[idx, 'avg_cards_per_90'] = avg_cards
            df.at[idx, 'avg_fouls_per_90'] = avg_fouls

            # Update cumulative stats
            cumulative_cards += row['total_cards']
            cumulative_fouls += row['fouls_committed']
            cumulative_minutes += row['game_minutes']

    return df

In [35]:
fix_players_clean = calculate_avg_cards_per_90(fix_players_clean_og)

In [37]:
fix_players_clean[fix_players_clean['player_name'] == "Antoine Griezmann"].sort_values('fixture_date')

,fixture_id,team_id,team_name,player_id,player_name,game_minutes,fouls_committed,total_cards,fixture_date,cumulative_cards,cumulative_fouls,cumulative_minutes,avg_cards_per_90,avg_fouls_per_90
333216,203340,530,Atletico Madrid,56,Antoine Griezmann,81.0,1,0,2015-08-22 18:30:00 UTC,0,0,0,0.000000,0.000000
74715,203353,530,Atletico Madrid,56,Antoine Griezmann,90.0,2,1,2015-08-30 18:30:00 UTC,0,0,0,0.000000,1.111111
911221,203360,530,Atletico Madrid,56,Antoine Griezmann,90.0,1,0,2015-09-12 18:30:00 UTC,0,0,0,0.526316,1.578947
724834,354987,530,Atletico Madrid,56,Antoine Griezmann,90.0,0,0,2015-09-15 18:45:00 UTC,0,0,0,0.344828,1.379310
549349,203370,530,Atletico Madrid,56,Antoine Griezmann,90.0,1,0,2015-09-19 18:30:00 UTC,0,0,0,0.256410,1.025641
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
724814,1208619,530,Atletico Madrid,56,Antoine Griezmann,90.0,2,0,2024-11-23 15:15:00 UTC,0,0,0,0.139748,0.665095
797161,1299057,530,Atletico Madrid,56,Antoine Griezmann,31.0,0,0,2024-11-26 17:45:00 UTC,0,0,0,0.139387,0.668540
333222,1208624,530,Atletico Madrid,56,Antoine Griezmann,64.0,0,0,2024-11-30 20:00:00 UTC,0,0,0,0.139263,0.667947
797173,1208630,530,Atletico Madrid,56,Antoine Griezmann,90.0,1,1,2024-12-08 20:00:00 UTC,0,0,0,0.139008,0.666724


In [ ]:
# TODO select top 22 players for each match by game minutes and add their avg_cards_per_90 and avg_fouls_per_90 as separate columns

In [ ]:
import pandas as pd


# Clean all data before 2020. jan. 1.
fix_clean["fixture_date"] = pd.to_datetime(fix_clean["fixture_date"]).dt.tz_localize(None)
start_date = pd.Timestamp("2020-01-01")
fix_clean = fix_clean[fix_clean["fixture_date"] >= start_date]


print(fix_clean.head())



    fixture_id        fixture_date       fixture_referee  league_id  \
0      1299262 2025-01-30 20:00:00     Match Not Started          3   
1       566789 2020-07-06 18:30:00  Felix Brych, Germany         78   
2       566788 2020-07-02 18:30:00             F. Zwayer         78   
28     1119329 2023-08-30 19:00:00             I. Kovacs          2   
29      946878 2022-09-14 19:00:00             I. Peljto          2   

              league_name  teams_home_id   teams_home_name  teams_away_id  \
0      UEFA Europa League            604  Maccabi Tel Aviv            212   
1              Bundesliga            180     FC Heidenheim            162   
2              Bundesliga            162     Werder Bremen            180   
28  UEFA Champions League            400     FC Copenhagen           3491   
29  UEFA Champions League            400     FC Copenhagen            536   

      teams_away_name  
0            FC Porto  
1       Werder Bremen  
2       FC Heidenheim  
28  Raków Częs

In [ ]:

count3 = fix_clean.shape[0]

# Keep only the 4 main leagues
top_leagues = ["La Liga", "Serie A", "Bundesliga", "Premier League"]
fix_clean = fix_clean[fix_clean["league_name"].isin(top_leagues)]

count4 = fix_clean.shape[0]
print(f"Number of rows before filtering: {count3}")
print(f"Number of rows after filtering: {count4}")

Number of rows before filtering: 19135
Number of rows after filtering: 7484


In [ ]:
count1 = fix_teams_clean.shape[0]

# Drop all unneccessary teams' datas
team_ids_to_keep = fix_players_clean["team_id"].unique()
fix_teams_clean = fix_teams_clean[fix_teams_clean["team_id"].isin(team_ids_to_keep)]
count2 = fix_teams_clean.shape[0]

print(f"Number of rows before filtering: {count1}")
print(f"Number of rows after filtering: {count2}")

Number of rows before filtering: 65220
Number of rows after filtering: 64693


In [ ]:
print(fix_players_clean.columns)
print(fix_teams_clean.columns)
print(fix_clean.columns)

Index(['fixture_id', 'team_id', 'team_name', 'player_id', 'player_name',
       'game_minutes', 'fouls_committed', 'total_cards', 'cumulative_cards',
       'cumulative_fouls', 'cumulative_minutes', 'avg_cards_per_90',
       'avg_fouls_per_90'],
      dtype='object')
Index(['fixture_id', 'team_id', 'team_name', 'fouls', 'yellow_cards',
       'red_cards'],
      dtype='object')
Index(['fixture_id', 'fixture_date', 'fixture_referee', 'league_id',
       'league_name', 'teams_home_id', 'teams_home_name', 'teams_away_id',
       'teams_away_name'],
      dtype='object')


In [ ]:

merged = fix_clean.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_home_id"]]
merged = merged.rename(columns={"fouls": "home_team_fouls", "yellow_cards": "home_team_yellow_cards", "red_cards": "home_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])

merged = merged.merge(fix_teams_clean, on="fixture_id", how="inner")
merged = merged[merged["team_id"] == merged["teams_away_id"]]
merged = merged.rename(columns={"fouls": "away_team_fouls", "yellow_cards": "away_team_yellow_cards", "red_cards": "away_team_red_cards"})
merged = merged.drop(columns=["team_id", "team_name"])



In [ ]:
pd.set_option('display.max_columns', 100)
merged[merged['fixture_id'] == 1208526]

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards
7387,1208526,2024-08-31 19:30:00,Mario Melero,140,La Liga,537,Leganes,798,Mallorca,11,1,0,17,3,0


In [ ]:
# Needed in the final table:
#fixture referee, league_name, teams_home_name, teams_away_name, avg_home_team_fouls, avg_home_team_yellow_cards, avg_home_team_red_cards, avg_away_team_fouls, avg_away_team_yellow_cards, avg_away_team_red_cards, [avg_total_cards_player_{1-22}, avg_total_fouls_player_{1_22},] total_cards(yellow_cards+red_cards*2)


In [ ]:
# TODO 1 - DONE
# Meg kéne csinálni az átlagokat home és away csapatra is szabálytalanságokból, sárga és piros lapokból
# LEHET EGYSZERŰBB EGYBŐL TODO 2-VEL PRÓBÁLKOZNI!

# merged['avg_home_team_fouls'] = merged.groupby('teams_home_name')['home_team_fouls'].transform('mean')
# merged['avg_home_team_yellow_cards'] = merged.groupby('teams_home_name')['home_team_yellow_cards'].transform('mean')
# merged['avg_home_team_red_cards'] = merged.groupby('teams_home_name')['home_team_red_cards'].transform('mean')
# merged['avg_away_team_fouls'] = merged.groupby('teams_away_name')['away_team_fouls'].transform('mean')
# merged['avg_away_team_yellow_cards'] = merged.groupby('teams_away_name')['away_team_yellow_cards'].transform('mean')
# merged['avg_away_team_red_cards'] = merged.groupby('teams_away_name')['away_team_red_cards'].transform('mean')

In [ ]:
merged

,fixture_id,fixture_date,fixture_referee,league_id,league_name,teams_home_id,teams_home_name,teams_away_id,teams_away_name,home_team_fouls,home_team_yellow_cards,home_team_red_cards,away_team_fouls,away_team_yellow_cards,away_team_red_cards
0,566789,2020-07-06 18:30:00,"Felix Brych, Germany",78,Bundesliga,180,FC Heidenheim,162,Werder Bremen,16,2,0,7,2,0
3,566788,2020-07-02 18:30:00,F. Zwayer,78,Bundesliga,162,Werder Bremen,180,FC Heidenheim,7,4,1,16,2,0
5,868032,2022-10-01 14:00:00,A. Madley,39,Premier League,40,Liverpool,51,Brighton,9,1,0,13,1,0
7,592286,2020-12-27 16:30:00,K. Friend,39,Premier League,40,Liverpool,60,West Brom,8,0,0,5,1,0
8,1035339,2023-12-17 16:30:00,M. Oliver,39,Premier League,40,Liverpool,33,Manchester United,13,2,0,8,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14214,878055,2022-10-29 12:00:00,Carlos del Cerro,140,La Liga,723,Almeria,538,Celta Vigo,10,4,0,7,0,1
14216,878215,2023-04-09 16:30:00,José Sánchez,140,La Liga,723,Almeria,532,Valencia,11,3,0,11,2,0
14218,877967,2022-08-27 20:00:00,Jesús Gil,140,La Liga,723,Almeria,536,Sevilla,18,5,0,11,8,0
14220,878075,2022-11-09 18:00:00,Alejandro Muñiz,140,La Liga,723,Almeria,546,Getafe,17,8,1,8,4,0


In [ ]:
# TODO 2 - DONE
# Az átlagokat úgy kéne megcsinálni hogy csak az azon meccs előtti infóknak legyen az átlaga

avgs = ['home_team_fouls', 'home_team_yellow_cards', 'home_team_red_cards', 'away_team_fouls', 'away_team_yellow_cards', 'away_team_red_cards']
merged = merged.sort_values(by=["teams_home_id", "fixture_date"])
for avg in avgs:
    merged[f"avg_{avg}"] = merged.groupby("teams_home_id")[avg] \
    .expanding().mean().shift(1).reset_index(level=0, drop=True).fillna(0)

merged["avg_home_team_cards"] = merged["avg_home_team_yellow_cards"] + merged["avg_home_team_red_cards"] * 2
merged["avg_away_team_cards"] = merged["avg_away_team_yellow_cards"] + merged["avg_away_team_red_cards"] * 2

merged["total_cards"] = merged["home_team_yellow_cards"] + merged["away_team_yellow_cards"] + merged["home_team_red_cards"] * 2 + merged["away_team_red_cards"] * 2

final_cols = ["fixture_referee", "league_name", "teams_home_name", "teams_away_name", "avg_home_team_fouls", "avg_away_team_fouls","avg_home_team_cards","avg_away_team_cards", "total_cards"]
merged = merged[final_cols]
merged



,fixture_referee,league_name,teams_home_name,teams_away_name,avg_home_team_fouls,avg_away_team_fouls,avg_home_team_cards,avg_away_team_cards,total_cards
8832,"Anthony Taylor, England",Premier League,Manchester United,Norwich,0.000000,0.000000,0.000000,0.000000,0
4203,"Jonathan Moss, England",Premier League,Manchester United,Burnley,18.000000,7.000000,0.000000,0.000000,2
453,"Paul Tierney, England",Premier League,Manchester United,Wolves,14.500000,7.000000,0.000000,1.000000,5
8816,"Martin Atkinson, England",Premier League,Manchester United,Watford,14.333333,9.666667,1.000000,1.333333,1
8882,"Mike Dean, England",Premier League,Manchester United,Manchester City,12.250000,11.000000,0.750000,1.250000,6
...,...,...,...,...,...,...,...,...,...
1304,F. La Penna,Serie A,Monza,AS Roma,11.756098,13.658537,2.317073,2.731707,4
1312,A. Rapuano,Serie A,Monza,Venezia,11.857143,13.642857,2.333333,2.690476,8
5261,E. Feliciani,Serie A,Monza,AC Milan,11.976744,13.697674,2.418605,2.674419,3
5265,A. Colombo,Serie A,Monza,Lazio,12.113636,13.727273,2.409091,2.636364,6
